## 실험 내용
1. WalkOpt를 적용하였을 때 총비용이 많이 오르는 이유 확인하기

### WalkOpt를 적용한 경우

In [2]:
%load_ext autoreload
%autoreload 2

from util_0804_9 import *

import pandas as pd

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_10.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0), (0.5, -2, 1)]
temp_process_time = -1
for weight1, weight2, weight3 in weight_grid:
    bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

    bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
    cost = sum((bundle.cost for bundle in bundles)) / K

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -3 2772.5129999999995
1 -1 2672.967
0 -0.5 2628.9749999999995
0.5 -2 2690.795


### WalkOpt를 적용하지 않은 경우

In [4]:
%load_ext autoreload
%autoreload 2

from util_0804_7 import *

import pandas as pd

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_10.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0), (0.5, -2, 1)]
temp_process_time = -1
for weight1, weight2, weight3 in weight_grid:
    bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

    bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
    cost = sum((bundle.cost for bundle in bundles)) / K

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -3 2666.7309999999998
1 -1 2587.2619999999997
0 -0.5 2557.297499999999
0.5 -2 2618.0624999999995


In [5]:
min_init_cost_bundles

[Bundle(all_orders, CAR, [12, 3, 62, 79], [12, 3, 79, 62], 138, True),
 Bundle(all_orders, CAR, [4, 13, 110, 7], [13, 7, 4, 110], 110, True),
 Bundle(all_orders, CAR, [5, 174, 66, 182], [66, 5, 174, 182], 113, True),
 Bundle(all_orders, CAR, [46, 6, 48, 108, 33], [6, 33, 46, 48, 108], 135, True),
 Bundle(all_orders, BIKE, [170, 192, 188, 134], [170, 134, 188, 192], 100, True),
 Bundle(all_orders, CAR, [9, 78, 113, 102, 64], [102, 64, 9, 78, 113], 101, True),
 Bundle(all_orders, CAR, [22, 65, 10, 164], [10, 22, 65, 164], 103, True),
 Bundle(all_orders, CAR, [74, 61, 80, 11], [11, 80, 61, 74], 102, True),
 Bundle(all_orders, CAR, [42, 14, 83, 120], [42, 14, 83, 120], 127, True),
 Bundle(all_orders, CAR, [15, 20, 29, 144], [20, 15, 29, 144], 176, True),
 Bundle(all_orders, CAR, [86, 132, 16, 97, 92], [86, 132, 16, 97, 92], 143, True),
 Bundle(all_orders, CAR, [17, 18, 56, 118, 148], [18, 17, 56, 118, 148], 119, True),
 Bundle(all_orders, CAR, [19, 41, 128, 67], [41, 67, 19, 128], 104, Tru

### WalkOpt를 적용한 경우 디버깅

In [7]:
%load_ext autoreload
%autoreload 2

from util_0804_12 import *

import pandas as pd

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_10.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0), (0.5, -2, 1)]
temp_process_time = -1
for weight1, weight2, weight3 in weight_grid:
    bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

    bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
    cost = sum((bundle.cost for bundle in bundles)) / K

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -3 2666.7309999999998
1 -1 2587.2619999999997
0 -0.5 2557.297499999999
0.5 -2 2618.0624999999995
